# Calculating calibration densities and plotting models at UCB
- This workbook shows how we calculate densities for calibration data, plot models and calculate the cali lines

In [1]:
# Import useful python packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import math
import os
import datetime

## Downloading DiadFit locally
- The easiest way to install diad fit is by removing the #. It will install from PyPI
- You only need to do this once on each computer, or run in order to grab an updated version. 

### This imports DiadFit, and gives you the version. In all communication with Penny, make sure you say the version!


In [2]:
import DiadFit as pf
pf.__version__

'0.0.60'

## Import the data (after fitting and pressure matching)

In [6]:
merged_df=pd.read_excel(r"P:\WORK-GENERAL\POSTDOC-UCB\BERKELEY-VIBE\Documents\Instruments and calibrations\Gas Cell calibrations\Densimeter_fitting\Input_files\June1st_June2nd_Combo_PTranducer.xlsx").drop(columns={'Unnamed: 0','Unnamed: 0.1'})
#caldata['filename_merge']=caldata['filename_nocrr'].str.split(" ").str.get(1)
merged_df.head()

,Corrected_Splitting,Corr_Split+1σ,Corr_Split-1σ,filename_x,Splitting,Split_err_abs,Split_err_quadrature,Diad1_Combofit_Cent,Diad1_cent_err,Diad1_Combofit_Height,...,duration_s,start_time_P,end_time_P,median_pressure,mad_pressure,median_temp,mad_temp,ThermistorT_Av,ThermistorT_Std,AvP_Manuel
0,104.317427,104.319067,104.315787,12 C6_CRR_DiadFit,104.569330,0.002040,0.001474,1282.899792,0.001234,8503.996689,...,121,2023-06-01 12:14:40.050,2023-06-01 12:16:41.050,10.0988,0.001109,31.3,2.486900e-14,37.01100,0.000707,10.09850
1,104.253899,104.255586,104.252212,14 C7_CRR_DiadFit,104.506344,0.002089,0.001527,1283.055183,0.001320,4159.640332,...,121,2023-06-01 12:29:19.110,2023-06-01 12:31:20.110,9.4587,0.000416,31.5,0.000000e+00,37.00125,0.003031,9.45820
2,104.191284,104.193021,104.189548,16 C8,104.444319,0.002222,0.001583,1283.206389,0.001248,2439.022384,...,121,2023-06-01 12:45:40.210,2023-06-01 12:47:41.210,9.0964,0.000834,31.8,2.486900e-14,37.00100,0.003391,9.09595
3,104.187881,104.189711,104.186050,17 C9,104.441140,0.002336,0.001687,1283.204984,0.001410,2470.540943,...,121,2023-06-01 12:50:58.240,2023-06-01 12:52:59.240,9.1028,0.000600,31.8,2.486900e-14,37.00975,0.003419,9.10230
4,104.039708,104.041630,104.037786,19 C10,104.293657,0.002524,0.001787,1283.523492,0.001324,2413.101156,...,121,2023-06-01 13:15:49.360,2023-06-01 13:17:50.360,8.6394,0.000121,32.2,3.688281e-02,37.00225,0.004146,8.63880


In [7]:
merged_df['rho_gmL']=pf.calculate_rho_for_P_T(P_kbar=np.array(merged_df['median_pressure']/100),
                                               T_K=np.array(merged_df['ThermistorT_Av']+273.15), EOS='SW96')

merged_df.to_excel(r"P:\WORK-GENERAL\POSTDOC-UCB\BERKELEY-VIBE\Documents\Instruments and calibrations\Gas Cell calibrations\Densimeter_fitting\Input_files"+'/'+'merged_df_wrho_PWdata.xlsx')
merged_df.head()

,Corrected_Splitting,Corr_Split+1σ,Corr_Split-1σ,filename_x,Splitting,Split_err_abs,Split_err_quadrature,Diad1_Combofit_Cent,Diad1_cent_err,Diad1_Combofit_Height,...,start_time_P,end_time_P,median_pressure,mad_pressure,median_temp,mad_temp,ThermistorT_Av,ThermistorT_Std,AvP_Manuel,rho_gmL
0,104.317427,104.319067,104.315787,12 C6_CRR_DiadFit,104.569330,0.002040,0.001474,1282.899792,0.001234,8503.996689,...,2023-06-01 12:14:40.050,2023-06-01 12:16:41.050,10.0988,0.001109,31.3,2.486900e-14,37.01100,0.000707,10.09850,0.687860
1,104.253899,104.255586,104.252212,14 C7_CRR_DiadFit,104.506344,0.002089,0.001527,1283.055183,0.001320,4159.640332,...,2023-06-01 12:29:19.110,2023-06-01 12:31:20.110,9.4587,0.000416,31.5,0.000000e+00,37.00125,0.003031,9.45820,0.652035
2,104.191284,104.193021,104.189548,16 C8,104.444319,0.002222,0.001583,1283.206389,0.001248,2439.022384,...,2023-06-01 12:45:40.210,2023-06-01 12:47:41.210,9.0964,0.000834,31.8,2.486900e-14,37.00100,0.003391,9.09595,0.621071
3,104.187881,104.189711,104.186050,17 C9,104.441140,0.002336,0.001687,1283.204984,0.001410,2470.540943,...,2023-06-01 12:50:58.240,2023-06-01 12:52:59.240,9.1028,0.000600,31.8,2.486900e-14,37.00975,0.003419,9.10230,0.621495
4,104.039708,104.041630,104.037786,19 C10,104.293657,0.002524,0.001787,1283.523492,0.001324,2413.101156,...,2023-06-01 13:15:49.360,2023-06-01 13:17:50.360,8.6394,0.000121,32.2,3.688281e-02,37.00225,0.004146,8.63880,0.548014
